In [26]:
from sage.all import *
from hashlib import shake_256

# Define constants
ELEMENTSIZE_IN_BYTES = 16  # Example value, you should set this to the correct value for your context
NUMBER_OF_ROUND_CONSTANTS_N = 4
NUMBER_OF_ROUNDS = 7  # Example number of rounds, adjust as needed

def shake256_digest(msg, digest_size):
    shake = shake_256()
    shake.update(msg.encode())
    return shake.digest(digest_size)

def ZZ_p_from_bytes(bytes, n, prime):
    return sum(int(bytes[i]) * (256**i) for i in range(n)) % prime

def initialize_round_constants(rc, convert, msg, prime):
    # Generate byte sequence
    digest = shake256_digest(msg, len(rc) * ELEMENTSIZE_IN_BYTES)
    
    # Convert bytes to elements of T and store them into rc.
    for i in range(len(rc)):
        rc[i] = convert(digest[i*ELEMENTSIZE_IN_BYTES:(i+1)*ELEMENTSIZE_IN_BYTES], ELEMENTSIZE_IN_BYTES, prime)

# Example usage for GF(p)
def test_ZZ_p():
    # We generated a random prime of approximately 128 bits.
    prime = 258439831533290445326983084816294483837
    Fp = GF(prime)
    
    # Generate round constants
    rcs_n = [Fp(0)] * (NUMBER_OF_ROUNDS * 4)
    msg = "GF(" + str(prime) + ")"
    initialize_round_constants(rcs_n, ZZ_p_from_bytes, msg, prime)
    
    print("The case GF(p):")
    print("Round constants:")
    for i in range(len(rcs_n)):
        round_number = i // 4
        round_index = i % 4
        print(f"Round {round_number}, RC{round_index+1}: {rcs_n[i]}")
        
    return rcs_n

def round_function(a_prev, b_prev, c_prev, RC):
    M = Matrix([
    [0, 0, 1],
    [1, RC[3], RC[3]],
    [0, 1, 1]
        ])
    
    # 定义输入向量
    v = vector([a_prev, b_prev, c_prev + a_prev * b_prev])
    
    # 定义常数向量
    C = vector([RC[2], RC[0], RC[1]])
    
    # 计算输出
    output = M * v + C
    return output;

def encryption_algorithm(initial_state, round_constants):
    state = vector(initial_state)
    for i in range(NUMBER_OF_ROUNDS):
        RC = round_constants[i*4:(i+1)*4]
        state = round_function(state[0], state[1], state[2], RC)
    return state

# Generate round constants
rcs_n = test_ZZ_p()

# Initial state (example values)
prime = 258439831533290445326983084816294483837
Fp = GF(prime)
initial_state = [Fp(0), Fp(251789242753917490270138186129835616781), Fp(74985816126660450637941313818502149746)]

# Encrypt
final_state = encryption_algorithm(initial_state, rcs_n)
print(f"Final state: {final_state}")


The case GF(p):
Round constants:
Round 0, RC1: 68541230898875564184433587386530472773
Round 0, RC2: 243996978352647607255455871849543958527
Round 0, RC3: 216242002757749709702814460003466193472
Round 0, RC4: 161104092254758748862892244004647913846
Round 1, RC1: 239107158142029720340681351913345531458
Round 1, RC2: 45072316712161305293914974679709302392
Round 1, RC3: 32196226902436383379323869467897418797
Round 1, RC4: 120640346008196295970941331879771409396
Round 2, RC1: 239502052916712098154950045034899365773
Round 2, RC2: 228150815315558075020672625273072480350
Round 2, RC3: 47206210755916442036779190755975287135
Round 2, RC4: 211755999388554467784439519138358568609
Round 3, RC1: 60983030723641225517385581889293726593
Round 3, RC2: 101146291815875327491158354324957155288
Round 3, RC3: 49581284193169782943582676767175830973
Round 3, RC4: 67947846601375863055057797742187367086
Round 4, RC1: 128781171440623593294758807278146953342
Round 4, RC2: 80309933382728113941369314634333013404
Rou

In [24]:
#7轮3分支
prime = 258439831533290445326983084816294483837
F = GF(prime)
RC01 = 68541230898875564184433587386530472773
RC02 = 243996978352647607255455871849543958527
RC03 = 216242002757749709702814460003466193472
RC04 = 161104092254758748862892244004647913846

RC11 = 239107158142029720340681351913345531458
RC12 = 45072316712161305293914974679709302392
RC13 = 32196226902436383379323869467897418797
RC14 = 120640346008196295970941331879771409396

RC21 = 239502052916712098154950045034899365773
RC22 = 228150815315558075020672625273072480350
RC23 = 47206210755916442036779190755975287135
RC24 = 211755999388554467784439519138358568609

RC31 = 60983030723641225517385581889293726593
RC32 = 101146291815875327491158354324957155288
RC33 = 49581284193169782943582676767175830973
RC34 = 67947846601375863055057797742187367086

RC41 = 128781171440623593294758807278146953342
RC42 = 80309933382728113941369314634333013404
RC43 = 3242240713907755872089283645962177016
RC44 = 172207317930891953715656888860780698047

RC51 = 110647455917240633792044346323507675824
RC52 = 162624383539388457270573709567395707475
RC53 = 164002545354703704361229049449252909337
RC54 = 69271573659676022485055316469461209240

RC61 = 11146844273971815183962107890218718237
RC62 = 241029799888881147441238643694188703743
RC63 = 29978774153572335666551064831823281925
RC64 = 45135030694429640172203976694500806277
# 建立方程求解密钥

#在输入消息块的位置引入变量
R.<x0,x1> = PolynomialRing(F, order = 'degrevlex')
a61 = x1
a60 = F(0)
a62 = F(0)
a63 = a61 - RC61
a64 = a62 - RC62
a65 = a60 - RC63
a66 = a63 - a64 * RC64
a67 = a64 - a65
a68 = a66 * a67
a69 = a65 - a68

a50 = a67 - RC51
a51 = a69 - RC52
a52 = a66 - RC53
a53 = a50 - a51 * RC54
a54 = a51 - a52
a55 = a53 * a54
a56 = a52 - a55

a40 = a54 - RC41
a41 = a56 - RC42
a42 = a53 - RC43
a43 = a40 - a41 * RC44
a44 = a41 - a42
a45 = a43 * a44

a30 = a44 - RC31
a32 = a43 - RC33
a34 = x0
a33 = a34 + a32
a35 = a30 - a33 * RC34
a31 = a33
a46 = a31 + RC32
a36 = a35 * a34
a37 = a32 - a36

a20 = a34 - RC21
a21 = a37 - RC22
a22 = a35 - RC23
a23 = a20 - a21 * RC24
a24 = a21 - a22
a25 = a23 * a24
a26 = a22 - a25

a10 = a24 - RC11
a11 = a26 - RC12
a12 = a23 - RC13
a13 = a10 - a11 * RC14
a14 = a11 - a12
a15 = a13 * a14
a16 = a12 - a15

a00 = F(0)
a01 = a14 - RC01
a02 = a16 - RC02
a03 = a13 - RC03
a04 = a01 - a00
a05 = a04 / RC04
a06 = a05 - a03
a07 = a00 * a06
a08 = a03 - a07
def print_magma(Eqs, Vlist, magma_file):
    f = open(magma_file, 'w')
    f.write("Fp := GF({});\n".format(prime))
    f.write("R<" + ", ".join(Vlist) + "> := PolynomialRing(Fp, {}, \"lex\");\n".format(len(Vlist)))

    s = ""
    for i in range(len(Eqs)-1):
        s += "f{}, ".format(i)
    s += "f{}".format(len(Eqs)-1)    
    for i in range(len(Eqs)):        
        f.write("f{} := ".format(i) + str(Eqs[i]) + ";\n")

    f.write("I := ideal<R|" + s + ">;\n")
    f.write("time gb := GroebnerBasis(I : Al := \"FGLM\");\n")
    f.write("Variety(I);\n")
    f.close()
eqs = []
eqs.append(a46 + a45 - a42)
eqs.append(a05 - a02)
print_magma(eqs, ["x0", "x1"], "Cinimion.mag")
print(a06)
print(a08)

178395425698592275319523011103361373552*x0^2*x1^4 + 230929132450722674207608920083328848993*x0^3*x1^2 + 34593806466373195485212361468232094209*x0^2*x1^3 + 162714668851904661613543521528221572726*x0*x1^4 + 48856829968230786581556757958004175613*x0^4 + 113836056751503039113902571832901063143*x0^3*x1 + 216643717578239685237506338851606074992*x0^2*x1^2 + 106744942949945002344663061782251265592*x0*x1^3 + 196549195997358387216885415881978407857*x1^4 + 234974897999887001283878627103588017853*x0^3 + 58963617049102627297854299778733087975*x0^2*x1 + 14856157114957100868006346994258957392*x0*x1^2 + 102884832555105819689619505822212660661*x1^3 + 227203174954829815594849989292412284384*x0^2 + 173816535901454130659758802726296139100*x0*x1 + 73169190860731546597601861992458371240*x1^2 + 214080896127915498864952296122354743543*x0 + 63576312199270716894284250992617932069*x1 + 13087117390158927317085385574916875257
204715242877091803017006548405722778798*x0^2*x1^4 + 2284466552017519067796298055752153737

In [25]:
F = GF(prime)
R.<x0,x1,x2,x3> = PolynomialRing(F, order = 'degrevlex')
x0 = 187103619832894723262808939118657208618
x1 = 45850694686950099651317707533071110514
a00 = F(0)
a02 = F(178395425698592275319523011103361373552*x0^2*x1^4 + 230929132450722674207608920083328848993*x0^3*x1^2 + 34593806466373195485212361468232094209*x0^2*x1^3 + 162714668851904661613543521528221572726*x0*x1^4 + 48856829968230786581556757958004175613*x0^4 + 113836056751503039113902571832901063143*x0^3*x1 + 216643717578239685237506338851606074992*x0^2*x1^2 + 106744942949945002344663061782251265592*x0*x1^3 + 196549195997358387216885415881978407857*x1^4 + 234974897999887001283878627103588017853*x0^3 + 58963617049102627297854299778733087975*x0^2*x1 + 14856157114957100868006346994258957392*x0*x1^2 + 102884832555105819689619505822212660661*x1^3 + 227203174954829815594849989292412284384*x0^2 + 173816535901454130659758802726296139100*x0*x1 + 73169190860731546597601861992458371240*x1^2 + 214080896127915498864952296122354743543*x0 + 63576312199270716894284250992617932069*x1 + 13087117390158927317085385574916875257)
a04 = F(204715242877091803017006548405722778798*x0^2*x1^4 + 228446655201751906779629805575215373795*x0^3*x1^2 + 82022633007626637544526604635795263112*x0^2*x1^3 + 141655109022825162932550610830337231061*x0*x1^4 + 70351576196019603066434246723512043448*x0^4 + 71389227902315614001115213466559939718*x0^3*x1 + 173188142121947300027205316922760887198*x0^2*x1^2 + 186060425484293832702688765018811564121*x0*x1^3 + 42230209359724427023514935736044587567*x1^4 + 111729349974028302532308000023411585607*x0^3 + 89939794634313382709941105151642963162*x0^2*x1 + 78790339032601669299192843854024004905*x0*x1^2 + 168426018378382233435524874077836873392*x1^3 + 128102064476867937551521109189690702565*x0^2 + 235906453689347155678079505377271351182*x0*x1 + 227027155456948531668500099459755442820*x1^2 + 67464773301794688731648996417354055290*x0 + 113684587522642457306300015367118037260*x1 + 137277945985330566987568853787540183034)
print(a02)
print(a04)

251789242753917490270138186129835616781
74985816126660450637941313818502149746
